In [3]:
import pandas as pd
import numpy as np
import re
import os
import string
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score
#忽略警告
import warnings

#国家分类

In [14]:
#读取作者文件
data = pd.read_excel(r'C:\Users\15457\OneDrive\桌面\桌面文件\25.2.18-数据上传准备\1.预测金标准数据\2408 gender data.xlsx') #读取文件
   #可能读取的是csv格式，使用下面的代码
   #data = pd.read_csv(r'C:\Users\15457\OneDrive\桌面\桌面文件\25.2.18-数据上传准备\1.预测金标准数据\2408 gender data.csv')

#将需要预测的姓名列名称改为'need_pred_name'
data.rename(columns={'corresponding author full': 'need_pred_name'}, inplace=True)

#根据国家分类
grouped = data.groupby('last_reauthor_country')

# 清洗文件名中的非法字符
def clean_filename(filename):
    valid_chars = "-_.() %s%s" % (string.ascii_letters, string.digits)
    cleaned_filename = ''.join(c for c in filename if c in valid_chars)
    return cleaned_filename

# 遍历每个分组，并保存到CSV文件
output_path = r'C:\Users\15457\OneDrive\桌面\桌面文件\25.2.18-数据上传准备\1.预测金标准数据\1.按国家分类作者'
for need_pred_name, group in grouped:
    # 清洗国家名以避免文件保存错误
    cleaned_name = clean_filename(need_pred_name)
    filename = f'code_{cleaned_name}.csv'
    group.to_csv(os.path.join(output_path, filename), index=False)

#根据国家进行预测

In [21]:

#warnings.filterwarnings('ignore', category=pd.errors.PerformanceWarning)
warnings.filterwarnings('ignore')
# 定义概率阈值
confidence_level = 0.7

# 读取数据
data = pd.read_excel(
    r'C:\Users\15457\OneDrive\桌面\桌面文件\25.2.18-数据上传准备\1.预测金标准数据\3.预测国家列表\预测国家列表.xlsx')
data2 = pd.DataFrame(columns=['样本', '预测', '准确率'])


In [22]:

# 定义预处理函数
def preprocessor(x):
    x = "" if pd.isna(x) else str(x)
    x = re.sub('[^a-zA-Z]', '', x)
    return x.lower()


# 创建向量化器和分类器
vec = CountVectorizer(
    min_df=50,
    ngram_range=(2, 8),
    analyzer='char',
    preprocessor=lambda x: f'^{x.lower().replace(" ", "")},'
)

clf = LogisticRegression(max_iter=1000)
pipeline = Pipeline([('vectorizer', vec), ('logreg', clf)])


# 定义预测函数
def predict_gender(names):
    probas = pipeline.predict_proba(names)
    genders = []
    for proba in probas:
        if proba[0] > confidence_level:
            genders.append('F')
        elif proba[1] > confidence_level:
            genders.append('M')
        else:
            genders.append('')
    return np.array(genders)


In [23]:

# 循环处理每对样本和预测
# train_path为各个国家训练集所在的文件，格式为code_国家名.csv
for index, row in data.iterrows():
    co = row['样本']
    co2 = row['预测']

    print(f"Processing: Sample - {co}, Prediction - {co2}")

    train_path = f'C:\\Users\\15457\\OneDrive\\桌面\\桌面文件\\25.2.18-数据上传准备\\1.预测金标准数据\\2.训练样本-国家\\code_{co}.csv'
    file_path = f'C:\\Users\\15457\\OneDrive\\桌面\\桌面文件\\25.2.18-数据上传准备\\1.预测金标准数据\\1.按国家分类作者\\code_{co2}.csv'
    out_path = f'C:\\Users\\15457\OneDrive\\桌面\\桌面文件\\25.2.18-数据上传准备\\1.预测金标准数据\\置信度{confidence_level}\\pred_{co2}_{confidence_level}.csv'
    os.makedirs(os.path.dirname(out_path), exist_ok=True)

    # 检查训练数据文件是否存在
    if not os.path.exists(train_path):
        print(f"Training file not found for {co}. Skipping...")
        continue

    # 检查预测数据文件是否存在
    if not os.path.exists(file_path):
        print(f"Prediction file not found for {co2}. Skipping...")
        continue

    #判断是否存在同名文件
    if os.path.exists(out_path):
        print(f"File already exists for {co2}. Skipping...")
        continue

    # 读取训练数据
    df = pd.read_csv(train_path)
    df['name'].fillna('', inplace=True)
    df['gender'].fillna('', inplace=True)
    df = df[df.gender.isin(['M', 'F'])]

    X = df.name
    y = df.gender == 'M'

    # 分割训练集和测试集
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1)

    # 训练模型
    pipeline.fit(X_train, y_train)

    # 评估模型
    y_pred = predict_gender(X_test)
    mask = y_pred != ''
    y_test_filtered = y_test[mask]
    y_pred_filtered = y_pred[mask]
    y_pred_bool = (y_pred_filtered == 'M')
    accuracy = accuracy_score(y_test_filtered, y_pred_bool)
    print(f"Model accuracy: {accuracy}")

    new_row = pd.DataFrame({'样本': [co], '预测': [co2], '准确率': [accuracy]})
    data2 = pd.concat([data2, new_row], ignore_index=True)

    #判断是否存在同名文件
    if os.path.exists(out_path):
        print(f"File already exists for {co2}. Skipping...")
        continue

    # 预测
    dfrd = pd.read_csv(file_path)
    array1 = dfrd['need_pred_name'].fillna('')
    dfrd['pre gender'] = predict_gender(array1)

    # 保存结果
    dfrd.to_csv(out_path, index=False)

    print(f"Prediction saved to {out_path}")
    print("--------------------")

print("All processing completed.")
confidence_path = f'C:\\Users\\15457\\OneDrive\\桌面\\桌面文件\\25.2.18-数据上传准备\\1.预测金标准数据\\置信度_{confidence_level}_预测准确率.csv'
# 检查并创建父目录（如果不存在）
os.makedirs(os.path.dirname(confidence_path), exist_ok=True)
data2.to_csv(confidence_path, index=False)
print(f"Summary results saved {confidence_path}")


Processing: Sample - US, Prediction - USA
Prediction file not found for USA. Skipping...
Processing: Sample - CN, Prediction - CHINA
Model accuracy: 0.904746885349314
Prediction saved to C:\Users\15457\OneDrive\桌面\桌面文件\25.2.18-数据上传准备\1.预测金标准数据\置信度0.7\pred_CHINA_0.7.csv
--------------------
Processing: Sample - DE, Prediction - GERMANY
Model accuracy: 0.9188732394366197
Prediction saved to C:\Users\15457\OneDrive\桌面\桌面文件\25.2.18-数据上传准备\1.预测金标准数据\置信度0.7\pred_GERMANY_0.7.csv
--------------------
Processing: Sample - GB, Prediction - ENGLAND
Prediction file not found for ENGLAND. Skipping...
Processing: Sample - JP, Prediction - JAPAN
Model accuracy: 0.921146953405018
Prediction saved to C:\Users\15457\OneDrive\桌面\桌面文件\25.2.18-数据上传准备\1.预测金标准数据\置信度0.7\pred_JAPAN_0.7.csv
--------------------
Processing: Sample - FR, Prediction - FRANCE
Model accuracy: 0.913320736891828
Prediction saved to C:\Users\15457\OneDrive\桌面\桌面文件\25.2.18-数据上传准备\1.预测金标准数据\置信度0.7\pred_FRANCE_0.7.csv
--------------------

#整合预测数据

In [18]:
def merge_csv_files(folder_path, output_file):
    # 获取文件夹内所有的CSV文件
    csv_files = [f for f in os.listdir(folder_path) if f.endswith('.csv')]

    # 初始化一个空的DataFrame用于存储合并后的数据
    combined_df = pd.DataFrame()

    # 遍历所有CSV文件并合并
    for csv_file in csv_files:
        file_path = os.path.join(folder_path, csv_file)
        df = pd.read_csv(file_path)
        combined_df = pd.concat([combined_df, df], ignore_index=True)

    return combined_df

In [20]:

# 示例用法
folder_path = r'C:\Users\15457\OneDrive\桌面\桌面文件\25.2.18-数据上传准备\1.预测金标准数据\置信度0.7'  # 替换为你的文件夹路径
output_file = f'C:\\Users\\15457\\OneDrive\\桌面\\桌面文件\\25.2.18-数据上传准备\\1.预测金标准数据\\combined_df_confidence{confidence_level}.csv'  # 替换为你想保存的输出文件路径
combined_df = merge_csv_files(folder_path, output_file)
# 将合并后的数据保存到新的CSV文件中
combined_df.to_csv(output_file, index=False)
print(f"合并完成，结果已保存到 {output_file}")


合并完成，结果已保存到 C:\Users\15457\OneDrive\桌面\桌面文件\25.2.18-数据上传准备\1.预测金标准数据\combined_df_confidence0.7.csv
